# Project 1.2: Image Transformations
## Matrix Operations and Affine Transforms

**Module**: Mathematical Foundations  
**Project ID**: 1.2  
**Estimated Time**: 60 minutes  
**Prerequisites**: Project 1.1 (Array Operations), Basic trigonometry  

### Learning Objectives
By the end of this project, you will be able to:
- [ ] Understand matrix multiplication and its geometric interpretation
- [ ] Implement image rotation, scaling, and shearing
- [ ] Apply affine transformation matrices
- [ ] Visualize transformation effects on images

### Mathematical Concepts Covered
- **Transformation matrices**: Rotation, scaling, shearing
- **Matrix multiplication**: Combining transformations
- **Affine transformations**: Linear maps + translations
- **Coordinate systems**: Image coordinates vs transformed coordinates

---

## Abstract

This project introduces matrix transformations through practical image manipulation applications. You will implement rotation, scaling, and shearing operations using transformation matrices, understanding the geometric interpretation of matrix multiplication. Building upon the array operations from Project 1.1, this project demonstrates how linear algebra concepts apply to computer graphics and image processing. Through hands-on implementation and visualization, you will develop intuition for how matrices represent and compose geometric transformations.

In [ ]:
# Setup and imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import math

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

print("✓ Libraries imported successfully")

## 1. Introduction

### Problem Statement
Image transformations are fundamental to computer graphics, computer vision, and augmented reality. Understanding how to rotate, scale, and deform images using matrix operations is essential for:
- Photo editing software (Photoshop, GIMP)
- Video game graphics
- Medical image registration
- Document scanning and deskewing
- Data augmentation in machine learning

### Real-World Applications
- **Instagram filters**: Rotation and perspective correction
- **Autonomous driving**: Stabilizing camera feeds
- **Medical imaging**: Aligning CT/MRI scans
- **Document processing**: Correcting skewed scans

### Prerequisites Checklist
- [ ] Completed Project 1.1
- [ ] Understand basic trigonometry (sine, cosine)
- [ ] Familiar with matrix multiplication

---

## 2. Theoretical Background

### 2.1 Transformation Matrices

A transformation matrix $T$ maps coordinates from one space to another:
\[
\begin{bmatrix} x' \\ y' \end{bmatrix} = T \begin{bmatrix} x \\ y \end{bmatrix}
\]

#### Rotation Matrix
To rotate a point by angle $	heta$ (counterclockwise):
\[
R(\theta) = \begin{bmatrix} \cos\theta & -\sin\theta \\ \sin\theta & \cos\theta \end{bmatrix}
\]

#### Scaling Matrix
To scale by factors $s_x$ and $s_y$:
\[
S(s_x, s_y) = \begin{bmatrix} s_x & 0 \\ 0 & s_y \end{bmatrix}
\]

#### Shearing Matrix
To shear horizontally by factor $h$ and vertically by $v$:
\[
Sh(h, v) = \begin{bmatrix} 1 & h \\ v & 1 \end{bmatrix}
\]

### 2.2 Combining Transformations

Multiple transformations are combined through matrix multiplication. Note that order matters!
\[
T_{\text{total}} = T_n \times \cdots \times T_2 \times T_1
\]

### 2.3 Affine Transformations

To include translations, we use homogeneous coordinates:
\[
\begin{bmatrix} x' \\ y' \\ 1 \end{bmatrix} = \begin{bmatrix} a & b & c \\ d & e & f \\ 0 & 0 & 1 \end{bmatrix} \begin{bmatrix} x \\ y \\ 1 \end{bmatrix}
\]

Where $(c, f)$ is the translation vector.

## 3. Methodology

We will implement:
1. Basic transformation matrices
2. Image rotation with proper center handling
3. Combined transformation pipeline
4. Interactive transformation visualization

## 4. Implementation

### 4.1 Creating Transformation Matrices

In [ ]:
def get_rotation_matrix(theta_degrees):
    """
    Create a 2D rotation matrix.
    
    Parameters:
    -----------
    theta_degrees : float
        Rotation angle in degrees (counterclockwise)
        
    Returns:
    --------
    np.ndarray
        2x2 rotation matrix
    """
    theta = np.radians(theta_degrees)
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    
    R = np.array([
        [cos_theta, -sin_theta],
        [sin_theta, cos_theta]
    ])
    
    return R

def get_scaling_matrix(sx, sy=None):
    """
    Create a 2D scaling matrix.
    
    Parameters:
    -----------
    sx : float
        Scale factor in x direction
    sy : float, optional
        Scale factor in y direction (default: same as sx)
    """
    if sy is None:
        sy = sx
    
    S = np.array([
        [sx, 0],
        [0, sy]
    ])
    
    return S

def get_shearing_matrix(hx=0, hy=0):
    """
    Create a 2D shearing matrix.
    
    Parameters:
    -----------
    hx : float
        Horizontal shear factor
    hy : float
        Vertical shear factor
    """
    Sh = np.array([
        [1, hx],
        [hy, 1]
    ])
    
    return Sh

# Test the matrices
print("Rotation matrix (45°):")
print(get_rotation_matrix(45))
print("\nScaling matrix (2x, 0.5y):")
print(get_scaling_matrix(2, 0.5))
print("\nShearing matrix (hx=0.5, hy=0):")
print(get_shearing_matrix(0.5, 0))

### 4.2 Visualizing Transformations

Let's see how these matrices transform a unit square.

In [ ]:
def plot_transformation(matrix, title, original_color='blue', transformed_color='red'):
    """
    Visualize how a transformation matrix affects a unit square.
    """
    # Original unit square corners
    square = np.array([
        [0, 0],
        [1, 0],
        [1, 1],
        [0, 1],
        [0, 0]  # Close the square
    ]).T
    
    # Apply transformation
    transformed = matrix @ square
    
    # Plot
    plt.figure(figsize=(8, 8))
    plt.plot(square[0], square[1], 'o-', color=original_color, 
             linewidth=2, markersize=8, label='Original')
    plt.plot(transformed[0], transformed[1], 's-', color=transformed_color, 
             linewidth=2, markersize=8, label='Transformed')
    
    # Add corner labels
    for i, (x, y) in enumerate(square.T[:-1]):
        plt.text(x-0.1, y-0.1, f'P{i}', fontsize=10, color=original_color)
    
    for i, (x, y) in enumerate(transformed.T[:-1]):
        plt.text(x+0.05, y+0.05, f"P{i}'", fontsize=10, color=transformed_color)
    
    plt.grid(True, alpha=0.3)
    plt.axis('equal')
    plt.legend(fontsize=12)
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.axhline(y=0, color='k', linewidth=0.5)
    plt.axvline(x=0, color='k', linewidth=0.5)
    plt.show()
    
    print(f"Transformation matrix:\n{matrix}")
    print(f"\nDeterminant: {np.linalg.det(matrix):.3f}")

# Visualize different transformations
plot_transformation(get_rotation_matrix(45), "Rotation by 45°")
plot_transformation(get_scaling_matrix(2, 0.5), "Scaling (2x, 0.5y)")
plot_transformation(get_shearing_matrix(0.5, 0), "Horizontal Shear (0.5)")

### 4.3 Combining Transformations

Let's see how order matters when combining transformations.

In [ ]:
# Compare: Rotate then Scale vs Scale then Rotate
R = get_rotation_matrix(30)
S = get_scaling_matrix(1.5, 0.5)

# Rotate then Scale: S @ R
T1 = S @ R

# Scale then Rotate: R @ S
T2 = R @ S

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

square = np.array([[0, 0], [1, 0], [1, 1], [0, 1], [0, 0]]).T

# Plot 1: Rotate then Scale
transformed1 = T1 @ square
axes[0].plot(square[0], square[1], 'o-', label='Original', linewidth=2)
axes[0].plot(transformed1[0], transformed1[1], 's-', label='Rotate→Scale', linewidth=2)
axes[0].set_title('Rotate then Scale\n(S @ R)', fontweight='bold')
axes[0].axis('equal')
axes[0].grid(True, alpha=0.3)
axes[0].legend()

# Plot 2: Scale then Rotate
transformed2 = T2 @ square
axes[1].plot(square[0], square[1], 'o-', label='Original', linewidth=2)
axes[1].plot(transformed2[0], transformed2[1], 's-', label='Scale→Rotate', linewidth=2)
axes[1].set_title('Scale then Rotate\n(R @ S)', fontweight='bold')
axes[1].axis('equal')
axes[1].grid(True, alpha=0.3)
axes[1].legend()

plt.tight_layout()
plt.show()

print("Note: The results are different! Order matters in matrix multiplication.")

### 4.4 Image Rotation Implementation

Now let's apply these concepts to rotate an actual image.

In [ ]:
def rotate_image(image, angle_degrees, center=None):
    """
    Rotate an image by given angle.
    
    Parameters:
    -----------
    image : np.ndarray
        Input image (2D or 3D)
    angle_degrees : float
        Rotation angle in degrees
    center : tuple, optional
        (x, y) center of rotation (default: image center)
        
    Returns:
    --------
    np.ndarray
        Rotated image
    """
    h, w = image.shape[:2]
    
    if center is None:
        center = (w // 2, h // 2)
    
    # Create rotation matrix (for OpenCV-style transformation)
    theta = np.radians(angle_degrees)
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    
    # Rotation matrix with translation to handle center
    cx, cy = center
    
    # Create output image
    rotated = np.zeros_like(image)
    
    # For each pixel in output, find corresponding input
    for y in range(h):
        for x in range(w):
            # Translate to origin
            x_rel = x - cx
            y_rel = y - cy
            
            # Rotate (inverse rotation for backward mapping)
            x_rot = x_rel * cos_theta + y_rel * sin_theta
            y_rot = -x_rel * sin_theta + y_rel * cos_theta
            
            # Translate back
            x_src = int(round(x_rot + cx))
            y_src = int(round(y_rot + cy))
            
            # Check bounds and assign
            if 0 <= x_src < w and 0 <= y_src < h:
                rotated[y, x] = image[y_src, x_src]
    
    return rotated

# Create a simple test image
def create_test_image_for_transform(size=200):
    """Create a test image with distinct features."""
    image = np.ones((size, size), dtype=np.uint8) * 128
    
    # Add a rectangle
    image[50:150, 50:150] = 255
    
    # Add a cross
    image[100, 40:160] = 0
    image[40:160, 100] = 0
    
    return image

# Test rotation
test_img = create_test_image_for_transform(200)
rotated_45 = rotate_image(test_img, 45)
rotated_90 = rotate_image(test_img, 90)

# Display
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(test_img, cmap='gray')
axes[0].set_title('Original', fontweight='bold')
axes[0].axis('off')

axes[1].imshow(rotated_45, cmap='gray')
axes[1].set_title('Rotated 45°', fontweight='bold')
axes[1].axis('off')

axes[2].imshow(rotated_90, cmap='gray')
axes[2].set_title('Rotated 90°', fontweight='bold')
axes[2].axis('off')

plt.suptitle('Image Rotation Using Matrix Transformations', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("✓ Image rotation implemented successfully")

### 4.5 Image Scaling

In [ ]:
def scale_image(image, scale_x, scale_y=None):
    """
    Scale an image by given factors.
    
    Parameters:
    -----------
    image : np.ndarray
        Input image
    scale_x : float
        Scale factor in x direction
    scale_y : float, optional
        Scale factor in y direction (default: scale_x)
    """
    if scale_y is None:
        scale_y = scale_x
    
    h, w = image.shape[:2]
    new_h = int(h * scale_y)
    new_w = int(w * scale_x)
    
    scaled = np.zeros((new_h, new_w), dtype=image.dtype)
    
    for y in range(new_h):
        for x in range(new_w):
            # Map back to original
            x_src = int(x / scale_x)
            y_src = int(y / scale_y)
            
            if 0 <= x_src < w and 0 <= y_src < h:
                scaled[y, x] = image[y_src, x_src]
    
    return scaled

# Test scaling
scaled_up = scale_image(test_img, 1.5)
scaled_down = scale_image(test_img, 0.6)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(test_img, cmap='gray')
axes[0].set_title(f'Original\n{test_img.shape}', fontweight='bold')
axes[0].axis('off')

axes[1].imshow(scaled_up, cmap='gray')
axes[1].set_title(f'Scale 1.5x\n{scaled_up.shape}', fontweight='bold')
axes[1].axis('off')

axes[2].imshow(scaled_down, cmap='gray')
axes[2].set_title(f'Scale 0.6x\n{scaled_down.shape}', fontweight='bold')
axes[2].axis('off')

plt.suptitle('Image Scaling', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"Original size: {test_img.shape}")
print(f"Scaled up: {scaled_up.shape}")
print(f"Scaled down: {scaled_down.shape}")

### 4.6 Complete Transformation Pipeline

Let's create a class that can chain multiple transformations.

In [ ]:
class ImageTransformer:
    """
    A class for chaining image transformations.
    """
    
    def __init__(self, image):
        self.original = image.copy()
        self.transformed = image.copy()
        self.transformations = []
    
    def rotate(self, angle):
        """Apply rotation."""
        self.transformed = rotate_image(self.transformed, angle)
        self.transformations.append(f'Rotate({angle}°)')
        return self
    
    def scale(self, sx, sy=None):
        """Apply scaling."""
        self.transformed = scale_image(self.transformed, sx, sy)
        sy = sy if sy else sx
        self.transformations.append(f'Scale({sx}, {sy})')
        return self
    
    def reset(self):
        """Reset to original."""
        self.transformed = self.original.copy()
        self.transformations = []
        return self
    
    def show_comparison(self):
        """Show before and after."""
        fig, axes = plt.subplots(1, 2, figsize=(12, 5))
        
        axes[0].imshow(self.original, cmap='gray')
        axes[0].set_title('Original', fontweight='bold')
        axes[0].axis('off')
        
        axes[1].imshow(self.transformed, cmap='gray')
        transform_str = ' -> '.join(self.transformations)
        axes[1].set_title(f'Transformed\n{transform_str}', fontweight='bold')
        axes[1].axis('off')
        
        plt.tight_layout()
        plt.show()
        return self

# Test the transformer
transformer = ImageTransformer(test_img)
transformer.rotate(30).scale(1.2).show_comparison()

print("\n✓ ImageTransformer class working correctly")

## 5. Results

### Summary of Transformations

| Transformation | Matrix | Effect |
|----------------|--------|---------|
| Rotation | $\begin{bmatrix} \cos\theta & -\sin\theta \\ \sin\theta & \cos\theta \end{bmatrix}$ | Rotates around origin |
| Scaling | $\begin{bmatrix} s_x & 0 \\ 0 & s_y \end{bmatrix}$ | Stretches/compresses axes |
| Shearing | $\begin{bmatrix} 1 & h \\ v & 1 \end{bmatrix}$ | Slants the image |
| Combined | $T_n \times \cdots \times T_1$ | Sequential application |

### Key Observations
1. **Order matters**: Matrix multiplication is not commutative
2. **Determinant**: Indicates area scaling (|det| > 1 = expansion, < 1 = compression)
3. **Inverse transformations**: Can be achieved with inverse matrices

## 6. Discussion

### Key Findings
1. **Matrix representations**: All linear transformations can be represented as matrices
2. **Composability**: Complex transformations built from simple ones
3. **Backward mapping**: Essential for image transformations to avoid holes

### Limitations
- Simple nearest-neighbor interpolation (pixelated results)
- No anti-aliasing
- Slow implementation (nested loops)

### Improvements
- Bilinear/bicubic interpolation for smoother results
- Vectorized NumPy operations for speed
- GPU acceleration for real-time processing

## 7. Conclusion

You have successfully:
- Implemented rotation, scaling, and shearing matrices
- Applied transformations to images
- Built a transformation pipeline class
- Understood the importance of transformation order

**Next Steps**: Project 1.3 - Optimization Basics, where you'll learn gradient descent!

---

## Track Your Progress

In [ ]:
import sys
sys.path.append('../..')
from utils.progress_tracker import get_tracker

tracker = get_tracker()
tracker.mark_lesson_complete('module_1', 'project_1_2_image_transformations', xp_earned=50)

print("\n" + "="*50)
print("✓ PROJECT 1.2 COMPLETED!")
print("="*50)

stats = tracker.get_stats()
print(f"\n🏆 Total XP: {stats['user']['total_xp']}")
print(f"📊 Level: {stats['user']['current_level']}")